# Calibración y heurísticas
Este cuaderno documenta:
- `calibrate_weights_greedy`: selección por capa de (per-tensor/per-channel) × (simétrica/asimétrica) minimizando error local.
- `tune_percentiles_SA`: enfriamiento simulado para afinar percentiles de activación por capa.


In [2]:
# --- Setup rápido para importar src.* desde notebooks/ ---
import sys
from pathlib import Path

# Sube hasta encontrar la carpeta raíz del repo (la que contiene src/)
ROOT = Path.cwd()
while not (ROOT / "src").exists() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.insert(0, str(ROOT))
print("Ruta raíz añadida a sys.path →", ROOT)


Ruta raíz añadida a sys.path → c:\Users\josem\Desktop\EL AÑO\OH\ptq-int8-project


In [3]:
import inspect, textwrap
from src.calibrator import calibrate_weights_greedy, init_activation_qparams, tune_percentiles_SA

def show(fn):
    print(f"--- {fn.__module__}.{fn.__name__} ---")
    print(textwrap.dedent(inspect.getsource(fn)))

show(calibrate_weights_greedy)
show(init_activation_qparams)
show(tune_percentiles_SA)


--- src.calibrator.calibrate_weights_greedy ---
def calibrate_weights_greedy(m, Xcal, max_samples=256):
    X = Xcal[:max_samples]
    inter = forward_intermediates(m, X)
    best = {}
    for layer in ["c1","c2","fc1","fc2"]:
        W = getattr(m, layer).W
        candidates = [("per_channel", True), ("per_tensor", True), ("per_tensor", False)]
        best_err, best_q = 1e30, None
        for scheme, sym in candidates:
            q = fit_weight_qparams(W, scheme=scheme, symmetric=sym)
            if layer == "c1":
                qs = QSimConvReLU(W, getattr(m, layer).b, q,
                                  fit_act_qparams(inter["inp"]),  # in
                                  fit_act_qparams(inter["a1"]))   # out
                y_hat = qs.forward(inter["inp"]);  y_ref = inter["a1"]
            elif layer == "c2":
                qs = QSimConvReLU(W, getattr(m, layer).b, q,
                                  fit_act_qparams(inter["p1"]),   # <-- in correcto (post-pool)
            

## 1) Greedy por capa
Idea: para **cada capa** probamos combinaciones de esquema y simetría, cuantizamos pesos y medimos **error local** (p.ej. MSE entre salida FP32 y QSim de la capa con datos de calibración). Elegimos la que **minimiza** ese error.


In [4]:
import numpy as np
from src.quantizer import fit_weight_qparams, quantize_weights

# supongamos pesos sintéticos de una "capa"
W = np.random.randn(4, 4, 3, 3).astype(np.float32)

cands = [
    dict(scheme="per_tensor",  symmetric=True),
    dict(scheme="per_tensor",  symmetric=False),
    dict(scheme="per_channel", symmetric=True),
    dict(scheme="per_channel", symmetric=False),
]

def fake_layer_mse(W_fp32, W_q):
    # Ejemplo: error de reconstrucción de pesos como proxy (ilustrativo)
    return float(np.mean((W_fp32 - W_q.astype(np.float32))**2))

best, best_err = None, 1e9
for C in cands:
    qparams = fit_weight_qparams(W, scheme=C["scheme"], symmetric=C["symmetric"])
    Wq = quantize_weights(W, qparams)
    err = fake_layer_mse(W, Wq)
    if err < best_err:
        best, best_err = C, err

best, best_err


({'scheme': 'per_tensor', 'symmetric': True}, 2575.445068359375)

## 2) Percentiles de activación y SA
- Inicializamos percentiles (p.ej., p=99) por capa.
- Definimos un **vecindario** (p ± Δ).
- Con **enfriamiento** vamos aceptando candidatos que **mejoran** métrica global (validación) y, a temperatura alta, permitimos empeoramientos pequeños para escapar de óptimos locales.


In [5]:
import numpy as np

def metric(p):
    # métrica sintética (máximo cerca de p* ~ 97.5)
    return -((p - 97.5)**2) + 100

p = 99.0
T = 2.0
best = (p, metric(p))
rng = np.random.default_rng(0)
for it in range(20):
    cand = p + rng.normal(0, 0.7)   # vecino
    m_cur, m_cand = metric(p), metric(cand)
    if (m_cand > m_cur) or (np.exp((m_cand - m_cur)/max(T,1e-6)) > rng.random()):
        p = cand
    if metric(p) > best[1]:
        best = (p, metric(p))
    T *= 0.95

best  # p cercano a ~97.5 en esta demo


(97.4798070991393, 99.99959224675483)

## 3) Relación con la memoria
- §3 *Implementación*: Algoritmos de calibración (`CALIBRATE_WEIGHTS`, `CALIBRATE_ACTS_SA`) que aquí hemos comentado paso a paso.
- §4 *Experimentos*: el ajuste por SA (Tabla de ablación) explica la recuperación de precisión frente a p fijo.
